#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#import flights table
flights = pd.read_csv('flights.csv', low_memory=False)
flights.shape

(15927485, 42)

In [3]:
# select 1000 random rows to use for testing 
flights_sample = flights.sample(n=1000)

In [4]:
flights_sample = flights_sample[(flights_sample['dep_delay'].isna() != True)] 
flights_sample = flights_sample[(flights_sample['dep_delay'] < 1000)] 
flights_sample = flights_sample[(flights_sample['diverted']!=1) | (flights_sample['cancelled']!=1)]
flights_sample = flights_sample[(flights_sample['arr_delay'].isna() != True)] 
flights_sample = flights_sample[(flights_sample['air_time'].isna() != True)] 

In [43]:
top_airports = flights.groupby('origin')['fl_date'].count().reset_index(name='count').sort_values(['count'], ascending=False)
top_airports = top_airports.reset_index(drop=True)
top_10_airports = top_airports.head(10)

In [44]:
passengers = pd.read_csv('passengers.csv')

In [48]:
top_passengers = passengers.groupby('origin')['passengers'].sum().reset_index(name='count').sort_values(['count'], ascending=False)
top_passengers = top_passengers.reset_index(drop=True)
top_10_passengers = top_passengers.head(10)

### The airports with the highest numbers of flights do not also share the highest number of passengers.

In [49]:
top = pd.merge(top_airports, top_passengers, on='origin')
top = top.rename(columns={'count_x':'total_flights','count_y':'total_passengers'})
top10 = top.head(10)
top10

,origin,total_flights,total_passengers
0,ORD,790253,194019087
1,ATL,785850,257478059
2,DFW,584688,163888490
3,DEN,552134,151024252
4,CLT,501557,112838675
5,LAX,500840,205745255
6,SEA,379367,113960665
7,SFO,357626,133266076
8,PHX,355485,109444579
9,IAH,354379,103908868


In [51]:
flights_pct = (top_10_airports['count'].sum()/flights['fl_date'].count())*100
pass_pct = (top_10_passengers['count'].sum()/passengers['passengers'].sum())*100
busy_pct = (top10['total_passengers'].sum()/top['total_passengers'].sum())*100

print(f"The Top 10 Airports handle {round(flights_pct,1)}% of all flights and {round(busy_pct,1)}% of all passengers")
print(f"The Top 10 Passenger Airports handle {round(pass_pct,1)}% of all passengers")

The Top 10 Airports handle 32.4% of all flights and 35.5% of all passengers
The Top 10 Passenger Airports handle 32.4% of all passengers


In [36]:
top

,origin,total_flights,total_passengers
0,ORD,790253,194019087
1,ATL,785850,257478059
2,DFW,584688,163888490
3,DEN,552134,151024252
4,CLT,501557,112838675
5,LAX,500840,205745255
6,SEA,379367,113960665
7,SFO,357626,133266076
8,PHX,355485,109444579
9,IAH,354379,103908868
